In [1]:
import taichi as ti
import numpy as np
from typing import Tuple
import napari
import matplotlib.pyplot as plt

[Taichi] version 1.7.4, llvm 15.0.1, commit b4b956fd, win, python 3.11.5


In [2]:
from beamgen import EllipticalBeamGenerator
from defocus_microscope import DefocusMicroscope
from helpers import init_trajectories
from simulation import Simulation

In [3]:
N = 128
runner = Simulation(N)

[Taichi] Starting on arch=cuda


### Define volume

In [4]:
SPHERES = [
    # Sphere 1 (Low Intensity)
    {'center': ti.Vector([40.0, 75.0, 40.0]), 'radius': 30.0, 'radius2': 20.0, 'value': 0.5},
    # Sphere 2 (Medium Intensity)
    {'center': ti.Vector([88.0, 88.0, 40.0]), 'radius': 25.0, 'radius2': 10.0, 'value': 0.7},
    # Sphere 3 (High Intensity)
    {'center': ti.Vector([40.0, 88.0, 88.0]), 'radius': 20.0, 'radius2': 18.0, 'value': 1.0},
]

runner.generate_spheres(SPHERES)

### Generate photons and initialize tracked values

In [ ]:
# generate MC light packets
# Set up beam parameters
central_point = (64, 84, 64)  # Target at origin
distance = np.sqrt(2)*64+1  # Source 10 units away
theta = 0*np.pi/2  # 45 degrees from z-axis
phi = np.pi/2    # 30 degrees in xy-plane
mask_diameter = 1.5*np.sqrt(128**2)
sigma_a = 256
sigma_b = 256
divergence_a = -0.02
divergence_b = -0.02         # why is this sign reversed?
divergence_sigma_a = 0.005
divergence_sigma_b = 0.005

MAX_STEPS = round(distance * 10)

runner.init_beam_generator(central_point, distance, theta, phi, mask_diameter, sigma_a, sigma_b, divergence_a, divergence_b, divergence_sigma_a, divergence_sigma_b)
runner.init_photons(100000000)
runner.init_tracking(100, MAX_STEPS)


In [6]:
# Initialize the napari viewer
viewer = napari.Viewer(title="3D Test Volume", ndisplay=3)

### View volume and projections

In [7]:
projections = runner.project_volume(["x", "y", "z"])
proj0_3d = projections["z"]
proj1_3d = projections["y"]
proj2_3d = projections["x"]
volume_np = runner.volume_np()

In [8]:
viewer.add_image(
    volume_np, 
    name='3D Index', 
    colormap='turbo', 
    contrast_limits=[0, 1],
    rendering='average',
    blending='additive'
)

# add the same volume and render as plane
# plane should be in 'additive' blending mode or depth looks all wrong

sum0_layer = viewer.add_image(
    proj0_3d,
    rendering='average',
    name='sum0',
    depiction='plane',
    blending='additive',
    opacity=0.25,
    plane={'position': (0, N/2, N/2),'normal': (1, 0, 0)}
)

sum1_layer = viewer.add_image(
    proj1_3d,
    rendering='average',
    name='sum1',
    depiction='plane',
    blending='additive',
    opacity=0.25,
    plane={'position': (N/2, 0, N/2),'normal': (0, 1, 0)}
)

sum2_layer = viewer.add_image(
    proj2_3d,
    rendering='average',
    name='sum2',
    depiction='plane',
    blending='additive',
    opacity=0.25,
    plane={'position': (N/2, N/2, 0),'normal': (0, 0, 1)}
)

In [9]:
axes = np.array((
    [[0, 0, 0], [128,0,0]],
    [[0, 0, 0], [0, 128, 0]],
    [[0, 0, 0], [0, 0, 128]])
)
viewer.add_shapes(axes[0], shape_type='line', edge_width=0.2, edge_color='red', opacity=0.75, name='axis-0')
viewer.add_shapes(axes[1], shape_type='line', edge_width=0.2, edge_color='green', opacity=0.75, name='axis-1')
viewer.add_shapes(axes[2], shape_type='line', edge_width=0.2, edge_color='blue', opacity=0.75, name='axis-2')


<Shapes layer 'axis-2' at 0x25a33507e50>

### Field Gradient Calculation

In [10]:
"""

# calculate local gradients
gradient_mag = ti.field(dtype=ti.f32, shape=volume.shape)
gradient_dir = ti.Vector.field(3, dtype=ti.f32, shape=volume.shape)

@ti.kernel
def compute_gradient():
    h = ti.static(1)
    # Precompute normalization factor sigma = sum(ox^2) over ox * (2h+1)^2
    sigma = 0.0
    for ox in ti.static(range(-h, h + 1)):
        sigma += ox * ox
    sigma *= (2 * h + 1) ** 2
    #if sigma == 0.0:  # Avoid division by zero (though N>=3)
    #    return

    for i, j, k in volume:
        # Skip boundaries (you can add padding or mirroring if needed)
        if i < h or i >= nx - h or j < h or j >= ny - h or k < h or k >= nz - h:
            gradient_mag[i, j, k] = 0.0
            gradient_dir[i, j, k] = ti.Vector([0.0, 0.0, 0.0])
            continue

        gx = 0.0
        gy = 0.0
        gz = 0.0
        # Loop over neighborhood
        for ox in ti.static(range(-h, h + 1)):
            for oy in ti.static(range(-h, h + 1)):
                for oz in ti.static(range(-h, h + 1)):
                    f = volume[i + ox, j + oy, k + oz]
                    gx += ox * f
                    gy += oy * f
                    gz += oz * f

        gx /= sigma
        gy /= sigma
        gz /= sigma

        mag = ti.sqrt(gx * gx + gy * gy + gz * gz)
        gradient_mag[i, j, k] = mag

        if mag > 1e-6:
            gradient_dir[i, j, k] = ti.Vector([gx / mag, gy / mag, gz / mag])
        else:
            gradient_dir[i, j, k] = ti.Vector([0.0, 0.0, 0.0])
            gradient_mag[i, j, k] = 0.0

# Usage example
nx, ny, nz = 128, 128, 128
compute_gradient()  # Change to 5, 7, etc.
if False:
    mag_np = gradient_mag.to_numpy()
    dir_np = gradient_dir.to_numpy()
    viewer.add_image(
    mag_np, 
    name='3D Index', 
    colormap='red', 
    contrast_limits=[0, 1],
    rendering='average',
    blending='additive'
    )
    viewer.add_vectors(
        dir_np, 
        name='Gradient Dir', 
        edge_color='red'
    )
"""

"\n\n# calculate local gradients\ngradient_mag = ti.field(dtype=ti.f32, shape=volume.shape)\ngradient_dir = ti.Vector.field(3, dtype=ti.f32, shape=volume.shape)\n\n@ti.kernel\ndef compute_gradient():\n    h = ti.static(1)\n    # Precompute normalization factor sigma = sum(ox^2) over ox * (2h+1)^2\n    sigma = 0.0\n    for ox in ti.static(range(-h, h + 1)):\n        sigma += ox * ox\n    sigma *= (2 * h + 1) ** 2\n    #if sigma == 0.0:  # Avoid division by zero (though N>=3)\n    #    return\n\n    for i, j, k in volume:\n        # Skip boundaries (you can add padding or mirroring if needed)\n        if i < h or i >= nx - h or j < h or j >= ny - h or k < h or k >= nz - h:\n            gradient_mag[i, j, k] = 0.0\n            gradient_dir[i, j, k] = ti.Vector([0.0, 0.0, 0.0])\n            continue\n\n        gx = 0.0\n        gy = 0.0\n        gz = 0.0\n        # Loop over neighborhood\n        for ox in ti.static(range(-h, h + 1)):\n            for oy in ti.static(range(-h, h + 1)):\n  

### Photon Interaction

In [11]:
runner.run_simulation(MAX_STEPS, 0.5)

In [12]:
interactions_np = runner.get_interactions()
viewer.add_image(
    interactions_np, 
    name='3D Interactions', 
    colormap='turbo', 
    contrast_limits=[0, 1],
    rendering='average',
    blending='additive'
)

output_np = runner.get_exits()
viewer.add_image(
    output_np, 
    name = 'Exiting Light',
    colormap='turbo',
    contrast_limits=[0, 1],
    rendering='average',
    blending='additive'
)

<Image layer 'Exiting Light' at 0x25a3357a450>

### Create defocused microscope and save image projection

In [15]:
face = "+x"

runner.init_microscope(face, N, 1.0, 0.25, 20, (N, N), 64.0)

runner.to_numpy()

# Form image with defocus
image_defocus = runner.defocus_image(0)

# Debugging: print statistics
print(f"\nMicroscope image statistics:")
print(f"  Shape: {image_defocus.shape}")
print(f"  Min: {image_defocus.min():.6e}")
print(f"  Max: {image_defocus.max():.6e}")
print(f"  Mean: {image_defocus.mean():.6e}")
print(f"  Sum: {image_defocus.sum():.6e}")
print(f"  Non-zero pixels: {(image_defocus > 0).sum()}")

# Add to viewer as a plane at the correct face position
if image_defocus.max() > 0:
    # Map observation face to plane position and normal
    # NOTE: Positions should be within [0, N-1] bounds
    face_config = {
        '+x': {'position': (N/2, N/2, 0), 'normal': (0, 0, -1)},   # Changed N to N-1
        '-x': {'position': (N/2, N/2, 0), 'normal': (0, 0, 1)},
        '+z': {'position': (0, N/2, N/2), 'normal': (1, 0, 0)},   # Changed N to N-1
        '-z': {'position': (0, N/2, N/2), 'normal': (1, 0, 0)},
        '+y': {'position': (N/2, 0, N/2), 'normal': (0, -1, 0)},   # Changed N to N-1
        '-y': {'position': (N/2, 0, N/2), 'normal': (0, 1, 0)},
    }

    config = face_config[face]
    
    # Reshape image to 3D for plane rendering (same as projection planes)
    if face in ['+z', '-z']:
        # z-face: image is in xy plane
        image_3d = image_defocus[np.newaxis, :, :]  # Shape: (1, Y, X)
    elif face in ['+x', '-x']:
        # x-face: image is in yz plane
        image_3d = image_defocus[:, :, np.newaxis]  # Shape: (Y, 1, Z)
    elif face in ['+y', '-y']:
        # y-face: image is in xz plane
        image_3d = image_defocus[:, np.newaxis, :]  # Shape: (X, Z, 1)
    
    viewer.add_image(
        image_3d,
        name=f'Microscope Defocus ({face})',
        colormap='gray',
        contrast_limits=[0, image_defocus.max()],
        rendering='average',
        depiction='plane',
        blending='additive',  # Changed to additive like projection planes
        opacity=0.8,
        plane={
            'position': config['position'],
            'normal': config['normal']
        }
    )
    print(f"\nAdded defocus microscope image to viewer at face {face}")
    print(f"  Plane position: {config['position']}")
    print(f"  Image 3D shape: {image_3d.shape}")
    print(f"  Intensity range: [{image_defocus.min():.2e}, {image_defocus.max():.2e}]")
else:
    print("No signal in defocus microscope image")


Taichi Microscope with Defocus initialized:
  Observation face: +x
  Focal depth: 64.0 voxels (64.0 μm)
  Depth of field: 4.40 μm (4.40 voxels)
  NA: 0.25, Acceptance angle: 10.8°
  Min sigma (in-focus): 1.75 pixels
Face +x: 125284 photons at face, 485 within NA
  Unscattered photons: 0 (0.0%)
  Sigma range: [1.75, 12.81] pixels
  Peak intensity: 1.67e-05

Microscope image statistics:
  Shape: (128, 128)
  Min: 0.000000e+00
  Max: 1.673385e-05
  Mean: 6.103515e-07
  Sum: 1.000000e-02
  Non-zero pixels: 3721

Added defocus microscope image to viewer at face +x
  Plane position: (64.0, 64.0, 0)
  Image 3D shape: (128, 128, 1)
  Intensity range: [0.00e+00, 1.67e-05]


### Multi-microscope funcitonality

In [ ]:
# Optional: Create microscope images from all 6 faces as planes
create_all_views = False  # Set to True to generate all views

if create_all_views:
    faces = ['-x', '-y']
    microscope_images = {}
    
    # Map observation face to plane position and normal
    # NOTE: Positions should be within [0, N-1] bounds
    face_config = {
        '+x': {'position': (N/2, N/2, N-1), 'normal': (0, 0, -1)},   # Changed N to N-1
        '-x': {'position': (N/2, N/2, 0), 'normal': (0, 0, 1)},
        '+z': {'position': (N-1, N/2, N/2), 'normal': (-1, 0, 0)},   # Changed N to N-1
        '-z': {'position': (0, N/2, N/2), 'normal': (1, 0, 0)},
        '+y': {'position': (N/2, N-1, N/2), 'normal': (0, -1, 0)},   # Changed N to N-1
        '-y': {'position': (N/2, 0, N/2), 'normal': (0, 1, 0)},
    }
    
    for face in faces:
        print(f"\n--- Creating microscope for face {face} ---")
        microscope = DefocusMicroscope(
            observation_face=face,
            volume_size=N,
            voxel_size=1.0,
            NA=0.25,
            magnification=10,
            sensor_size=(N, N),  # Match volume size
            focal_depth=N/2  # Focus at center
        )
        
        image = microscope.form_image_with_defocus(
            positions_np, directions_np, intensities_np,
            entered_np, exited_np, last_scatter_pos_np
        )
        
        microscope_images[face] = image
        
        # Add to napari as plane at the correct face
        if image.max() > 0:
            config = face_config[face]
            
            # Reshape image to 3D for plane rendering (same as projection planes)
            if face in ['+z', '-z']:
                image_3d = image[np.newaxis, :, :]  # (1, Y, X)
            elif face in ['+x', '-x']:
                image_3d = image[:, np.newaxis, :]  # (Y, 1, Z)
            elif face in ['+y', '-y']:
                image_3d = image[:, :, np.newaxis]  # (X, Z, 1)
            
            viewer.add_image(
                image_3d,
                name=f'Microscope {face}',
                colormap='gray',
                contrast_limits=[0, 0.01],
                rendering='average',
                depiction='plane',
                blending='additive',  # Changed to additive like projection planes
                opacity=0.7,
                plane={
                    'position': config['position'],
                    'normal': config['normal']
                },
                visible=True
            )
    
    print("\n=== Summary ===")
    for face, img in microscope_images.items():
        print(f"{face}: {img.sum():.2e} total intensity, max: {img.max():.2e}")